In [1]:
# Install necessary dependencies
!pip install ultralytics

import os
from ultralytics import YOLO

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
# Define paths to the data directories
train_images_dir = '/content/drive/My Drive/train/images'
train_labels_dir = '/content/drive/My Drive/train/labels'
valid_images_dir = '/content/drive/My Drive/valid/images'
valid_labels_dir = '/content/drive/My Drive/valid/labels'


In [3]:
import matplotlib.pyplot as plt
import cv2

In [4]:
# Create YAML file for the dataset
dataset_yaml_content = f"""
path: /content/drive/My Drive

train: train/images
val: valid/images

nc: 5  # number of classes
names: ['car', 'bus', 'ambulance', 'motorcycle', 'other']  # class names
augment:
  flipud: 0.5  # Randomly flip images vertically
  fliplr: 0.5  # Randomly flip images horizontally
  hsv_h: 0.015  # Image HSV-Hue augmentation (fraction)
  hsv_s: 0.7  # Image HSV-Saturation augmentation (fraction)
  hsv_v: 0.4  # Image HSV-Value augmentation (fraction)
  translate: 0.1  # Image translation (fraction)
  scale: 0.5  # Image scale (fraction)
  shear: 0.0  # Image shear (fraction)
  perspective: 0.0  # Image perspective (fraction)
  rotate: 0.0  # Image rotation (degrees)
  mosaic: 1.0  # Mosaic augmentation probability
  mixup: 0.5  # Mixup augmentation probability
  cutmix: 0.5  # CutMix augmentation probability
"""

In [5]:
# Save the YAML file
yaml_file_path = '/content/drive/My Drive/dataset.yaml'
with open(yaml_file_path, 'w') as file:
    file.write(dataset_yaml_content)

In [6]:
# Define the custom Focal Loss function
import torch
import torch.nn as nn
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce_loss = nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        ce_loss = self.ce_loss(logits, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

In [7]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 320MB/s]


In [8]:
# Initial Training Phase
model.train(data=yaml_file_path, epochs=10)

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

100%|██████████| 755k/755k [00:00<00:00, 87.0MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      10/10      2.38G     0.8679      1.282      1.226         20        640: 100%|██████████| 55/55 [00:05<00:00,  9.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.77it/s]


                   all        250        454      0.725      0.536      0.594      0.451

10 epochs completed in 0.030 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.35it/s]


                   all        250        454      0.727       0.54      0.594      0.451
                   car         50         64      0.835      0.789      0.868      0.743
                   bus         30         46      0.806      0.543      0.617      0.511
             ambulance         90        238      0.557      0.479      0.476      0.335
            motorcycle         42         46      0.702      0.522      0.569       0.32
                 other         38         60      0.733      0.367       0.44      0.347
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7828f329a860>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [9]:
training_params = {
    'data': yaml_file_path,
    'epochs': 50,  # Adjust based on your needs
    'batch': 16,  # Adjust based on your GPU capabilities
    'imgsz': 640,  # Image size; larger images may yield better results
    'lr0': 0.01,  # Starting learning rate; fine-tune this parameter
    'optimizer': 'SGD',  # Experiment with different optimizers like 'Adam'
    'patience': 10,  # Early stopping patience to prevent overfitting
    'weight_decay': 0.0005,  # Regularization term to reduce overfitting
    'project': '/content/drive/My Drive/yolo_training',  # Save logs and weights to Google Drive
    'name': 'yolov8_experiment',  # Experiment name
    'save_period': 10  # Save weights every 10 epochs
}

In [10]:
# Train the model on the dataset
model.train(**training_params)

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/dataset.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/drive/My Drive/yolo_training, name=yolov8_experiment8, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/yolo_training/yolov8_experiment8/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/yolo_training/yolov8_experiment8
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        250        454      0.407      0.431      0.368      0.231
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



11 epochs completed in 0.035 hours.
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_experiment8/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_experiment8/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/yolo_training/yolov8_experiment8/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.31it/s]


                   all        250        454      0.705      0.502      0.574      0.433
                   car         50         64      0.758      0.812      0.863      0.733
                   bus         30         46      0.745      0.522      0.583      0.463
             ambulance         90        238      0.673      0.387       0.49      0.324
            motorcycle         42         46      0.683      0.391      0.561      0.369
                 other         38         60      0.664        0.4      0.373      0.278
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training/yolov8_experiment8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x782b5da63e50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [11]:
# Define the directory path
save_dir = '/content/drive/My Drive/yolov8_experiment'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model after training
model.save(os.path.join(save_dir, 'best_model.pt'))

In [12]:
# Step 3: Load the saved model for further improvement
model = YOLO('/content/drive/My Drive/yolov8_experiment/best_model.pt')


In [13]:
# Example: Fine-tuning with a lower learning rate and additional epochs
fine_tuning_params = {
    'data': yaml_file_path,  # Use the same or updated dataset
    'epochs': 20,  # Additional epochs for fine-tuning
    'batch': 16,  # Keep or adjust batch size based on your GPU
    'imgsz': 640,  # Image size; you can keep it the same or adjust
    'lr0': 0.005,  # Lower the learning rate for fine-tuning
    'optimizer': 'AdamW',  # Experiment with a different optimizer
    'patience': 10,  # Early stopping to prevent overfitting
    'weight_decay': 0.0005,  # Regularization to prevent overfitting
    'project': '/content/drive/My Drive/yolo_training',  # Same project path
    'name': 'yolov8_fine_tuning_experiment',  # New experiment name
    'save_period': 5,  # Save weights every 5 epochs
}

# Continue training the loaded model with new parameters
model.train(**fine_tuning_params)

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/My Drive/yolov8_experiment/best_model.pt, data=/content/drive/My Drive/dataset.yaml, epochs=20, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/drive/My Drive/yolo_training, name=yolov8_fine_tuning_experiment26, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment26/labels.jpg... 
optimizer: AdamW(lr=0.005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment26
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        250        454      0.599      0.511      0.534      0.397



20 epochs completed in 0.057 hours.
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment26/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment26/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment26/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.46it/s]


                   all        250        454      0.607      0.523      0.541      0.409
                   car         50         64      0.749      0.875      0.875      0.746
                   bus         30         46      0.803      0.609      0.624      0.467
             ambulance         90        238      0.537      0.429      0.438      0.306
            motorcycle         42         46      0.548      0.391      0.461      0.301
                 other         38         60      0.399       0.31      0.308      0.227
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment26


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7828f3580fd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [14]:
# Define the directory path
save_dir = '/content/drive/My Drive/yolov8_experiment'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model after training
model.save(os.path.join(save_dir, 'model#2.pt'))

In [15]:
# Step 3: Load the saved model for further improvement
model = YOLO('/content/drive/My Drive/yolov8_experiment/model#2.pt')



In [16]:
dataset_yaml_content = f"""
path: {save_dir}

train: train/images
val: valid/images

nc: 5  # number of classes
names: ['car', 'bus', 'ambulance', 'motorcycle', 'other']  # class names

# Augmentation strategies (emphasizing trucks and other classes)
augment:
  flipud: 0.5  # Flip vertically with a probability of 50%
  fliplr: 0.5  # Flip horizontally with a probability of 50%
  hsv_h: 0.015  # HSV Hue augmentation (range: 0-0.5)
  hsv_s: 0.7  # HSV Saturation augmentation (range: 0-1.0)
  hsv_v: 0.4  # HSV Value augmentation (range: 0-1.0)
  translate: 0.1  # Translate images (range: 0-0.5)
  scale: 0.5  # Scale images (range: 0.5-2.0)
  shear: 0.1  # Shear images (range: 0-0.5)
  perspective: 0.1  # Apply perspective transformation (range: 0-0.5)
  rotate: 10.0  # Rotate images by 10 degrees
  mosaic: 1.0  # Use Mosaic augmentation with 100% probability
  mixup: 0.5  # Increase Mixup augmentation to create harder examples
  cutout: 0.5  # Use Cutout to remove parts of the image and force learning
  'class_weights': [1, 1.5, 1.5, 2.5, 3],  # Further increase weights for Ambulance and Other classes
  'cos_lr': True,  # Implement Cosine Annealing Learning Rate
"""

# Save the YAML file again with updated augmentations
yaml_file_path = os.path.join(save_dir, 'dataset_improved.yaml')
with open(yaml_file_path, 'w') as file:
    file.write(dataset_yaml_content)

In [17]:
# Continue training the loaded model with the new parameters and augmented dataset
model.train(**fine_tuning_params)

# Save the fine-tuned model after additional training
model.save(os.path.join(save_dir, 'fine_tuned_model_final.pt'))

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/My Drive/yolov8_experiment/model#2.pt, data=/content/drive/My Drive/dataset.yaml, epochs=20, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/drive/My Drive/yolo_training, name=yolov8_fine_tuning_experiment27, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment27/labels.jpg... 
optimizer: AdamW(lr=0.005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment27
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.43G     0.8179     0.8324      1.228         29        640: 100%|██████████| 55/55 [00:05<00:00,  9.34it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.21it/s]

                   all        250        454      0.632      0.508      0.534      0.398



20 epochs completed in 0.057 hours.
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment27/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment27/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment27/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.32it/s]


                   all        250        454      0.646      0.476      0.542      0.408
                   car         50         64      0.689      0.781      0.855      0.739
                   bus         30         46      0.792      0.565       0.64      0.518
             ambulance         90        238      0.621       0.42      0.449      0.295
            motorcycle         42         46      0.709      0.348      0.495      0.288
                 other         38         60      0.417      0.267      0.272        0.2
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment27
